In [20]:
import cv2
import numpy as np
import yaml

In [21]:
frame = cv2.imread("frame_0.png", cv2.IMREAD_COLOR)
frame_size = (frame.shape[1], frame.shape[0])


In [22]:
#Coordinates of quadrangle vertices in the source image.
src = np.float32([
                [450,230],
                [600,312],
                [80,312],
                [281,230]])

#Coordinates of the corresponding quadrangle vertices in the destination image.
dst = np.float32([
        [450,230],
        [450,312],
        [281,312],
        [281,230]])

In [23]:
H = cv2.getPerspectiveTransform(src, dst)
warped = cv2.warpPerspective(frame, H, frame_size, flags=cv2.INTER_LINEAR)

In [24]:
# create indices of the destination image and linearize them
h, w = warped.shape[:2]
indy, indx = np.indices((h, w), dtype=np.float32)
lin_homg_ind = np.array([indx.ravel(), indy.ravel(), np.ones_like(indx).ravel()])

In [72]:
map_ind = H.dot(lin_homg_ind)
map_x, map_y = map_ind[:-1]/map_ind[-1]
map_x = map_x.reshape(h, w).astype(np.float32) 
map_y = map_y.reshape(h, w).astype(np.float32)
#print (map_x)

In [ ]:
# remap!
dst = cv2.remap(frame, map_x, map_y, cv2.INTER_LINEAR)
blended = cv2.addWeighted(warped, 0.5, dst, 0.5, 0)
cv2.imshow('blended.png', blended)
cv2.waitKey()
cv2.destroyAllWindows()

In [64]:
fname = "remap.yaml"
data = {"remap_ipm_x":{"rows":frame.shape[1],"cols":frame.shape[0],"data":map_x.tolist()},"remap_ipm_y":{"rows":frame.shape[1],"cols":frame.shape[0],"data":map_y.tolist()}}

with open(fname, "w") as f:
    yaml.dump(data, f)

In [74]:
ipm_remap_file_path ='./tusimple_ipm_remap.yml'
#ipm_remap_file_path ='./remap.yaml'
print (ipm_remap_file_path)

./tusimple_ipm_remap.yml


In [75]:
_ipm_remap_file_path = ipm_remap_file_path

In [76]:

fs = cv2.FileStorage(_ipm_remap_file_path, cv2.FILE_STORAGE_READ)

remap_to_ipm_x = fs.getNode('remap_ipm_x').mat()
remap_to_ipm_y = fs.getNode('remap_ipm_y').mat()

print (remap_to_ipm_x)

[[ 5.6235349e-01  2.8119411e+00  5.0597892e+00 ...  1.1745839e+03
   1.1760929e+03  1.1776011e+03]
 [-5.6425714e-01  1.6892282e+00  3.9409683e+00 ...  1.1751602e+03
   1.1766710e+03  1.1781810e+03]
 [-1.6943864e+00  5.6300980e-01  2.8186553e+00 ...  1.1757379e+03
   1.1772505e+03  1.1787622e+03]
 ...
 [-8.5828500e+04 -8.2145703e+04 -7.8745586e+04 ...  3.1267615e+03
   3.1318381e+03  3.1368994e+03]
 [-1.0566682e+05 -1.0021692e+05 -9.5272289e+04 ...  3.1431902e+03
   3.1482761e+03  3.1533469e+03]
 [-1.3730392e+05 -1.2836803e+05 -1.2047805e+05 ...  3.1598455e+03
   3.1649412e+03  3.1700212e+03]]


In [77]:
print (map_x)

[[456.0653  455.8581  455.65085 ... 324.05872 323.85147 323.64426]
 [456.4784  456.27008 456.06177 ... 323.77527 323.56693 323.3586 ]
 [456.89587 456.68643 456.47702 ... 323.4888  323.27936 323.06995]
 ...
 [325.7143  325.8521  325.98993 ... 413.50253 413.64035 413.77814]
 [325.8954  326.0327  326.17007 ... 413.37827 413.5156  413.65292]
 [326.07523 326.2121  326.34897 ... 413.25485 413.3917  413.52856]]
